In [1]:
import findspark
import os
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
# SPARK_HOME='/home/qiany/.conda/envs/py37'
# os.environ['SPARK_HOME'] = '/home/qiany/.conda/envs/py37'
findspark.init(SPARK_HOME)

In [2]:
import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
#from hdfs import InsecureClient
import numpy as np
#from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
#import igraph
#from igraph import Graph
import geofeather
from pyspark.storagelevel import StorageLevel

In [3]:
tin_file = input("Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file:")
print("\n********************")

# get the directory to the TIN file
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or SigSpatial(1):")
if directory_type == '0':
    directory = 'hdfs_data'
else:
    directory = 'SigSpatial_data'
    
# get the basename to the TIN file
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

# get the filename of the TIN file
tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

# get the type of TIN file: off, tri, etc
tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)
print("\n********************")

Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file: cos_sum.tri



********************
tin_directory:  


Is the data stored in hdfs(0) or SigSpatial(1): 0


tin_basename:  cos_sum.tri
tin_filename:  cos_sum
tin_extension:  .tri

********************


In [4]:
# filtra = input("Do you have filtration data?")
filtra = 'yes'

# if filtra.lower() == 'no':
#     Basic_Data = input("Do you have basic pts and tri data?")
    
Num_executor = input("spark.executor.instances:")
Num_core_per_executor = input("spark.executor.cores:")
Memory_executor = input("spark.executor.memory? Please end with 'g':")
MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':")

# Num_core_per_driver = Num_core_per_executor
# Memory_driver = Memory_executor
# MemoryOverhead_driver = MemoryOverhead_executor
Num_core_per_driver = '5'
Memory_driver = '12g'
MemoryOverhead_driver = '4g'

Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")
# StorageFraction = input("spark.memory.storageFraction:")
# Num_subgraphs = int(input("How many subgraphs:"))

# Num_executor = '8'
# Num_core_per_executor = '5'
# Memory_executor = '7g'
# MemoryOverhead_executor = '1g'
# Num_shuffle_partitions = '120'
# StorageFraction = '0.5'

spark.executor.instances: 4
spark.executor.cores: 5
spark.executor.memory? Please end with 'g': 10g
spark.executor.memoryOverhead? Please end with 'g': 2g
spark.sql.shuffle.partitions: 20


In [5]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [6]:
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [7]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''
# .config('spark.memory.storageFraction', StorageFraction) \

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "V2_M3_BFS_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
.config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '20') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.getOrCreate()

spark_app_name: V2_M3_BFS_cos_sum_03072024_1459


In [8]:
# Register function is essential for Apache Sedona Core and Apache Sedona SQL. 
SedonaRegistrator.registerAll(spark)

True

In [9]:
spark.sparkContext.defaultParallelism

20

In [10]:
import sys
sys.path.append("/local/data/yuehui/pyspark/FormanGradient/code")

In [11]:
import graphframes
from graphframes import GraphFrame
from graphframes import *
from graphframes.lib import Pregel

In [12]:
t_tri_order_0 = time.time()

In [13]:
# StructType is a collection of StructField’s which is used to define the column name, data type, and a flag for nullable or not.
# StructType: https://sparkbyexamples.com/pyspark/pyspark-sql-types-datatype-with-examples/

if filtra.lower() == 'yes':
    # create a DataFrame for original vertices
    schema_ver_origin = StructType([ \
        StructField("x",FloatType(),True), \
        StructField("y",FloatType(),True), \
        StructField("ele",FloatType(),True), \
        StructField("self_index",IntegerType(),True), \
        StructField("self_order",IntegerType(),True) \
      ])
    
    # hdfs_tin_pts_origin = "SigSpatial_data" + "/" + tin_filename + '_filtra_pts_origin.csv'
    hdfs_tin_pts_origin = "hdfs_data" + "/" + tin_filename + '_filtra_pts_origin.csv'

    df_ver_order = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_ver_origin)\
          .load(hdfs_tin_pts_origin)
    df_ver_order.printSchema()
    print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())

    
    # create DataFrames for original triangles
    schema_tri_origin = StructType([ \
        StructField("v1",IntegerType(),True), \
        StructField("v2",IntegerType(),True), \
        StructField("v3",IntegerType(),True), \
        StructField("tri_order",IntegerType(),True) \
      ])
    
    # hdfs_tin_tri_origin = "SigSpatial_data" + "/" + tin_filename + '_filtra_tri_origin.csv'
    hdfs_tin_tri_origin = "hdfs_data" + "/" + tin_filename + '_filtra_tri_origin.csv'
    
    df_tri_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tri_origin)\
          .load(hdfs_tin_tri_origin)
    df_tri_origin.printSchema()
    print("Number of partitions for df_tri_origin:", df_tri_origin.rdd.getNumPartitions())

root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: integer (nullable = true)
 |-- self_order: integer (nullable = true)

Number of partitions for df_ver_order: 1
root
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- tri_order: integer (nullable = true)

Number of partitions for df_tri_origin: 1


In [14]:
if filtra.lower() == 'no':
    # StructType is a collection of StructField’s which is used to define the column name, data type, and a flag for nullable or not.
    # StructType: https://sparkbyexamples.com/pyspark/pyspark-sql-types-datatype-with-examples/
    schema_ver_origin = StructType([ \
        StructField("x",FloatType(),True), \
        StructField("y",FloatType(),True), \
        StructField("ele",FloatType(),True), \
        StructField("self_index",IntegerType(),True) \
      ])
    
    hdfs_tin_pts_origin = "SigSpatial_data" + "/" + tin_filename + '_pts_origin.csv'

    df_ver_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_ver_origin)\
          .load(hdfs_tin_pts_origin)
    
    print("Number of partitions:", df_ver_origin.rdd.getNumPartitions())
    # df_ver_origin.printSchema()
    
    # define a window for the ordering
    # row_number() function along with partitionBy() of other column populates the row number by group
    # Since we want to order the whole DataFrame, so we don't need the partitionBy() function
    w = Window().orderBy(col('ele').asc())
    df_ver_order = df_ver_origin.withColumn("self_order", F.row_number().over(w) -1) # let the row number start from 0
    # df_ver_order = df_ver_origin.withColumn("self_order", col("self_order").cast(IntegerType())) # change the data type of self_order to LongType
    
    # df_ver_order.printSchema()
    print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())

    
    # create DataFrames for original triangles
    schema_tri_origin = StructType([ \
        StructField("v1",IntegerType(),True), \
        StructField("v2",IntegerType(),True), \
        StructField("v3",IntegerType(),True), \
        StructField("tri_order",IntegerType(),True) \
      ])
    
    hdfs_tin_tri_origin = "SigSpatial_data" + "/" + tin_filename + '_tri_origin.csv'
    
    df_tri_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tri_origin)\
          .load(hdfs_tin_tri_origin)
    print("Number of partitions for df_tri_origin:", df_tri_origin.rdd.getNumPartitions())

In [15]:
# persist df_ver_order
t_persist_df_ver_order_0 = time.time()

# df_ver_order.persist(StorageLevel.MEMORY_ONLY_SER)

t_persist_df_ver_order_1 = time.time()
print("Total time cost for persisting df_ver_order:", t_persist_df_ver_order_1 - t_persist_df_ver_order_0)

Total time cost for persisting df_ver_order: 7.677078247070312e-05


In [14]:
# join two DataFrames by df_tri_origin.v1 == df_ver_order.self_index
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_tri_order_v1 = df_tri_origin.join(df_ver_order, df_tri_origin.v1 == df_ver_order.self_index, "inner")
df_tri_order_v1 = df_tri_order_v1.select(col("tri_order"), col("v1"), col("v2"), col("v3"),col("self_order").alias("r1"), col("ele").alias("r1_ele"))
df_tri_order_v1.printSchema()
print("The number of partitions for df_tri_order_v1:", df_tri_order_v1.rdd.getNumPartitions())
# df_tri_order_v1.show()

root
 |-- tri_order: integer (nullable = true)
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r1_ele: float (nullable = true)

The number of partitions for df_tri_order_v1: 20


In [15]:
# join two DataFrames by df_tri_order_v1.v2 == df_ver_order.self_index
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_tri_order_v2 = df_tri_order_v1.join(df_ver_order, df_tri_order_v1.v2 == df_ver_order.self_index, "inner")
df_tri_order_v2 = df_tri_order_v2.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("self_order").alias("r2"), col("r1_ele"), col("ele").alias("r2_ele"))
df_tri_order_v2.printSchema()
#df_tri_order_v2.show()

root
 |-- tri_order: integer (nullable = true)
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)



In [16]:
# join two DataFrames by df_tri_order_v1.v2 == df_ver_order.self_index
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_tri_order_v3 = df_tri_order_v2.join(df_ver_order, df_tri_order_v2.v3 == df_ver_order.self_index, "inner")
df_tri_order_v3 = df_tri_order_v3.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("r2"), col("self_order").alias("r3"), col("r1_ele"), col("r2_ele"), col("ele").alias("r3_ele"))
df_tri_order_v3.printSchema()
#df_tri_order_v3.show()

root
 |-- tri_order: integer (nullable = true)
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)



In [17]:
# the new triangles DataFrame 
df_tri_order = df_tri_order_v3.select(col("tri_order"), col("r1"), col("r2"), col("r3"), col("r1_ele"), col("r2_ele"), col("r3_ele"))
df_tri_order.printSchema()
print("The number of partitions for df_tri_order:", df_tri_order.rdd.getNumPartitions())
# df_tri_order.show()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)

The number of partitions for df_tri_order: 20


In [18]:
# sort tri in ascending order

'''
def get_tri_array(r1, r2, r3):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
    tri = [r1, r2, r3]    
    tri.sort(reverse=True)
    
    return tri

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_tri_array_udf = udf(get_tri_array, ArrayType(IntegerType()))
df_tri_order = df_tri_order.withColumn("tri", get_tri_array_udf(df_tri_order.r1, df_tri_order.r2, df_tri_order.r3))
'''

df_tri_order = df_tri_order.withColumn("tri_origin", F.array("r1", "r2", "r3")).withColumn("tri_ele_origin", F.array("r1_ele", "r2_ele", "r3_ele"))
df_tri_order = df_tri_order.withColumn("tri", sort_array("tri_origin", False)).drop("tri_origin") # sort_array("tri", False), False means descending order
df_tri_order = df_tri_order.withColumn("tri_ele", sort_array("tri_ele_origin", False)).drop("tri_ele_origin") # sort_array("tri", False), False means descending order

df_tri_order.printSchema()
df_tri_order.rdd.getNumPartitions()
#df_tri_order.show()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- tri_ele: array (nullable = false)
 |    |-- element: float (containsNull = true)



20

In [21]:
# unpersist df_ver_order
# df_ver_order.unpersist()

# persist df_tri_order

t_persist_0 = time.time()

# df_tri_order.persist(StorageLevel.DISK_ONLY)

t_persist_1 = time.time()
print("Total time cost for persisting df_tri_order:", t_persist_1 - t_persist_0)

Total time cost for persisting df_tri_order: 4.5299530029296875e-05


In [22]:
# num_df_ver_order = df_ver_order.count()
# print("number of rows in df_ver_order:", num_df_ver_order)

# num_df_tri_origin = df_tri_origin.count()
# print("number of rows in df_tri_origin:", num_df_tri_origin)

t_tri_order_1 = time.time()

time_basic_data = t_tri_order_1-t_tri_order_0
print("time for preparing basic data:", time_basic_data)

time for preparing basic data: 1.8372502326965332


In [19]:
# groupby and collect_set
df_tri_group1 = df_tri_order.groupBy('r1','r1_ele').agg(collect_set('r2').alias('multi_r2'),collect_set('r3').alias('multi_r3'), collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
# df_tri_group1 = df_tri_order.groupBy('r1').agg(collect_set('r1_ele').alias('r1_ele'), collect_set('r2').alias('multi_r2'),collect_set('r3').alias('multi_r3'), collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
df_tri_group1.printSchema()
# df_tri_group1.show()

root
 |-- r1: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- multi_r2: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r3: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)



In [20]:
# groupby and collect_set
df_tri_group2 = df_tri_order.groupBy('r2','r2_ele').agg(collect_set('r1').alias('multi_r1'),collect_set('r3').alias('multi_r3'), collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
# df_tri_group2 = df_tri_order.groupBy('r2').agg(collect_set('r2_ele').alias('r2_ele'), collect_set('r1').alias('multi_r1'),collect_set('r3').alias('multi_r3'), collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
df_tri_group2.printSchema()
# df_tri_group2.show()

root
 |-- r2: integer (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- multi_r1: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r3: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)



In [21]:
# groupby and collect_set
df_tri_group3 = df_tri_order.groupBy('r3', 'r3_ele').agg(collect_set('r1').alias('multi_r1'),collect_set('r2').alias('multi_r2'), collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
# df_tri_group3 = df_tri_order.groupBy('r3').agg(collect_set('r3_ele').alias('r3_ele'), collect_set('r1').alias('multi_r1'),collect_set('r2').alias('multi_r2'), collect_list('tri').alias('multi_tri'), collect_list('tri_ele').alias('multi_tri_ele'))
df_tri_group3.printSchema()
# df_tri_group3.show()

root
 |-- r3: integer (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- multi_r1: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r2: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)



In [22]:
# concatenate the three dataframe in rows
t0 = time.time()

union12 = df_tri_group1.union(df_tri_group2) # the title of union12 will be the same as df_sj_group1
union123 = union12.union(df_tri_group3) # the title of union123 will be the same as union12, so as df_sj_group1
union123.printSchema()
# union123.show()

t1 = time.time()
print("time:",t1-t0)

root
 |-- r1: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- multi_r2: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r3: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

time: 0.018166065216064453


In [23]:
# groupby the first index to find entire VP relation
# collect_set and collect_list: https://sparkbyexamples.com/spark/spark-collect-list-and-collect-set-functions/
t0 = time.time()

union123_group = union123.groupBy('r1','r1_ele').agg(collect_set('multi_r2').alias('multi_r2_set'),collect_set('multi_r3').alias('multi_r3_set'), collect_list('multi_tri').alias('multi_tri_list'), collect_list('multi_tri_ele').alias('multi_tri_ele_list'))
# union123_group = union123.groupBy('r1').agg(collect_set('r1_ele'), collect_set('multi_r2').alias('multi_r2_set'),collect_set('multi_r3').alias('multi_r3_set'), collect_list('multi_tri').alias('multi_tri_list'), collect_list('multi_tri_ele').alias('multi_tri_ele_list'))
union123_group.printSchema()
print("number of partitions for union123_group:", union123_group.rdd.getNumPartitions())
# union123_group.show()

t1 = time.time()
print("time:",t1-t0)

root
 |-- r1: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- multi_r2_set: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_r3_set: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: float (containsNull = true)

number of partitions for union123_group: 20
time: 0.8440008163452148


In [24]:
# obtain VV relation
def get_multi_pt_order(pt_list1, pt_list2):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
    pt_set = set()
    # pt_list.append(pt_self) # if we don't calculate roughness, we don't need self vertex
    for i in range(len(pt_list1)):
        pt_set.update(pt_list1[i]) # update() will add multiple elements to a set, while add() only add one element to a set
                
    for i in range(len(pt_list2)):
        pt_set.update(pt_list2[i])
    
    pt_list = sorted(pt_set) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return pt_list # directly return a list, e.g., [0, 1, 5, 6], the length of returned column list will be 12, incluing "[" and empty char

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_multi_pt_order_udf = udf(get_multi_pt_order, ArrayType(IntegerType()))

union123_group_new = union123_group.withColumn("multi_pts_order", get_multi_pt_order_udf(union123_group.multi_r2_set, union123_group.multi_r3_set))
union123_group_new = union123_group_new.select(col("r1").alias("Ver"), col("r1_ele").alias("Ver_ele"), col("multi_pts_order").alias("VV"), col("multi_tri_list"), col("multi_tri_ele_list"))
union123_group_new.printSchema()

# we can also use the built-in flatten,array_union, and then sort_array to obtain sorted VV, but it is not efficient
# union123_group_new = union123_group.withColumn("multi_pts_order", array_union(flatten(col("multi_r2_set")), flatten(col("multi_r3_set"))))
# union123_group_new = union123_group_new.select(col("r1").alias("Ver"), col("r1_ele").alias("Ver_ele"), sort_array("multi_pts_order", True).alias("VV"), col("multi_tri_list"), col("multi_tri_ele_list"))

root
 |-- Ver: integer (nullable = true)
 |-- Ver_ele: float (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_tri_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |-- multi_tri_ele_list: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: float (containsNull = true)



In [25]:
# obtain VT relation
def get_multi_tri_order(tri_list, tri_ele_list):
# get_multi_tri_order is used to obtain the adjacent triangles, the results are in ascending order, but the triangle itself is in descending
# e.g., [[3, 2, 1], [5, 2, 3], [5, 3, 1]]
# tri_list: an array of array of array of integer
# tri_ele_list: an array of array of array of float

    tri = []
    tri_ele = []
    # pt_list.append(pt_self) # if we don't calculate roughness, we don't need self vertex
    for i in range(len(tri_list)):
        for j in range(len(tri_list[i])):
            # if tri_list[i][j] not in tri: # we do not need to check if it exists in tri_list, because we can prove that each one is unique
            tri.append(tri_list[i][j]) # update() will add multiple elements to a set, while add() only add one element to a set   
            tri_ele.append(tri_ele_list[i][j])
    
    tri_copy = copy.deepcopy(tri) # deep copy tri
    tri.sort() # sort the list of array, e.g., tri=[[3, 2, 1], [5, 2, 3], [5, 3, 1]], after sorting, [[3, 2, 1], [5, 2, 3], [5, 3, 1]]
    
    tri_ele_sort = []
    # sort tri_ele according to the same rule as tri
    for i in range(len(tri)):
        index_in_tri_origin = tri_copy.index(tri[i])
        tri_ele_sort.append(tri_ele[index_in_tri_origin])
    
    return tri, tri_ele_sort

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html

# StructType for get_multi_tri_order        
get_multi_tri_order_schema = StructType([
    StructField("multi_tri_sort", ArrayType(ArrayType(IntegerType())),True), 
    StructField('multi_tri_ele_sort',ArrayType(ArrayType(FloatType())),True)
])

get_multi_tri_order_udf = udf(get_multi_tri_order, get_multi_tri_order_schema)

df_VV_VT = union123_group_new.withColumn("multi_tri_order", get_multi_tri_order_udf(union123_group.multi_tri_list, union123_group.multi_tri_ele_list))
df_VV_VT = df_VV_VT.select("Ver", "Ver_ele", "VV", col("multi_tri_order.multi_tri_sort").alias("VT_filtra"), col("multi_tri_order.multi_tri_ele_sort").alias("VT_ele"))
df_VV_VT.printSchema()
print("number of partitions for df_VV_VT:", df_VV_VT.rdd.getNumPartitions())
# df_VV_VT.show()

root
 |-- Ver: integer (nullable = true)
 |-- Ver_ele: float (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

number of partitions for df_VV_VT: 20


In [30]:
# num_df_VV_VT = df_VV_VT.count()
# print("number of rows in df_VV_VT:", num_df_VV_VT)

t_df_VV_VT = time.time()

time_VV_VT = t_df_VV_VT-t_tri_order_1
print("time for preparing VV and VT:", time_VV_VT)

time for preparing VV and VT: 2.1031179428100586


obtain the lower star of each vertex

In [26]:
# obtain edges of Lower Star
def get_LS_edge(Ver, VV):
# get_LS_edge is used to obtain the edges of LS in ascending order
# VV: an array of VV relation in ascending order
    LS_edge = []
    for i in range(len(VV)):
        if VV[i] < int(Ver):
            LS_edge.append([int(Ver),VV[i]])
        
    return LS_edge

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_LS_edge_udf = udf(get_LS_edge, ArrayType(ArrayType(IntegerType())))

df_LS_edge = df_VV_VT.withColumn("LS_edge", get_LS_edge_udf(df_VV_VT.Ver, df_VV_VT.VV))
df_LS_edge.printSchema()
print("number of partitions for df_LS_edge:", df_LS_edge.rdd.getNumPartitions())
#df_LS_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- Ver_ele: float (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)
 |-- LS_edge: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)

number of partitions for df_LS_edge: 20


In [27]:
# obtain triangle of Lower Star
def get_LS_tri(Ver, VT_filtra, VT_ele):
# get_LS_tri is used to obtain the triangles of LS in ascending order
# VT_filtra: an array of VT relation in ascending order
# VT_ele: elevation of corresponding triangles
    LS_tri = []
    LS_tri_ele = []
    
    for i in range(len(VT_filtra)):
        if Ver >= max(VT_filtra[i]): # Ver_ele is not the maximum vertex
            LS_tri.append(VT_filtra[i])
            LS_tri_ele.append(VT_ele[i])
                
    return LS_tri, LS_tri_ele

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
# StructType for get_multi_tri_order        
get_LS_tri_schema = StructType([
    StructField("LS_filtra", ArrayType(ArrayType(IntegerType())),True), 
    StructField('LS_ele',ArrayType(ArrayType(FloatType())),True)
])

get_LS_tri_udf = udf(get_LS_tri, get_LS_tri_schema)

df_LS_tri = df_LS_edge.withColumn("LS_tri", get_LS_tri_udf(df_LS_edge.Ver, df_LS_edge.VT_filtra, df_LS_edge.VT_ele))
df_LS_tri = df_LS_tri.select("Ver", "Ver_ele", "VV", "VT_filtra", "VT_ele", "LS_edge", col("LS_tri.LS_filtra").alias("LS_filtra"), col("LS_tri.LS_ele").alias("LS_ele"))

df_LS_tri.printSchema()
print("number of partitions for df_LS_tri:", df_LS_tri.rdd.getNumPartitions())
# df_LS_tri.show()

root
 |-- Ver: integer (nullable = true)
 |-- Ver_ele: float (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VT_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)
 |-- LS_edge: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_ele: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)

number of partitions for df_LS_tri: 20


In [33]:
# num_df_LS_tri = df_LS_tri.count()
# print("number of rows in df_LS_tri:", num_df_LS_tri)

t_df_LS_tri = time.time()

time_lower_star = t_df_LS_tri-t_df_VV_VT
print("time for preparing lower star:", time_lower_star)

time for preparing lower star: 1.6469717025756836


In [34]:
# df_LS_tri.select([count(c) for c in df_LS_tri.columns]).show()

compute Forman Gradient

In [28]:
from collections import deque

In [29]:
# Forman gradient
def get_Forman(Ver, LS_edge, LS_filtra, LS_ele):
# get_Forman is used to compute Forman gradient
# LS_edge is triangles of lower star in ascending order
# LS_tri is triangles of lower star in ascending order
    # inner udf
    def num_unpaired_edges(tri_inner, crit_cell_inner, Forman_vec_pair_VE_inner, Forman_vec_pair_ET_inner):
        # compute the number of unpaired edges
        edge0 = [tri_inner[0], tri_inner[1]]
        edge1 = [tri_inner[0], tri_inner[2]]
        
        num_unpaired_edge = 2
        paired_edge = []
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 2: # crit_inner_temp is a critical edge
                paired_edge.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_VE_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[1])
            # if type(pair_vec[0]) != int and len(pair_vec[0]) == 2: # pair_vec[0] is a paired edge, like in (edge, tri)
                # paired_edge.append(pair_vec[0])
            # if len(pair_vec[1]) == 2: # pair_vec[1] is a paired edge, like in (ver, edge)
                # paired_edge.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_ET_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, tri)
            paired_edge.append(pair_vec[0])
                
        if edge0 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
        if edge1 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            
        return num_unpaired_edge
    
    # inner udf
    def pair_an_edge_from_PQzero(alpha_inner, PQzero_inner):
    # alpha_inner is a triangle in [r0, r1, r2] format, and the vertext's elevation is in descending order
    # PQzero_inner is a queue storing all other edges
        alpha_e0 = [alpha_inner[0], alpha_inner[1]]
        alpha_e1 = [alpha_inner[0], alpha_inner[2]]
        for edge_inner in PQzero_inner:
            if edge_inner == alpha_e0:
                return edge_inner
            if edge_inner == alpha_e1:
                return edge_inner
            
    crit_cell = [] # crit_cell will store critical simplices
    # Forman_vec_pair = [] # Forman_vec_pair will store all Forman gradient pairs as a vector
    Forman_vec_pair_VE = []
    Forman_vec_pair_ET = []
    if (len(LS_edge)+len(LS_filtra)) == 0: # ver is a local minimum
        crit_cell.append([Ver]) # store minimum as an array [Ver] though it has only one element
    elif len(LS_edge) > 0:
        Forman_vec_pair_VE.append([Ver, LS_edge[0]]) # LS_edge[0] is the minimum edge since it is in ascending order
        
        # define two queues, PQzero storing all other edges, PQone storing all triangles which have only one unpaired edge
        PQzero = deque()
        PQone = deque()
        PQone_ele = deque()
        for i in range(1, len(LS_edge)): # add the other edges to PQzero
            PQzero.append(LS_edge[i])
            
        cell_temp_index_of_remove = []
        index_temp = 0
        for cell_temp in LS_filtra:
            if num_unpaired_edges(cell_temp, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                PQone.append(cell_temp) # storing cells with one unpaired edges, which are triangles in TIN
                PQone_ele.append(LS_ele[index_temp])
                # remove cell_temp from LS_filtra
                # LS_filtra.remove(cell_temp)
                cell_temp_index_of_remove.append(index_temp)
            index_temp += 1
            
        # remove cell_temp from LS_filtra and LS_ele
        for i in reversed(cell_temp_index_of_remove):
            del LS_filtra[i]
            del LS_ele[i]
                
        # sort PQone in ascending order, the initial queue is already sorted when creating
        # PQone = sorted(PQone) # sort() is not supported but we can use sorted()
        # PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
        while len(PQone) > 0 or len(PQzero) > 0:
            while len(PQone) > 0:
                # alpha = PQone.popleft() # alpha is a triangle in the PQone
                # alpha should be a triangle with lower elevation when PQone have more than one triangle
                if len(PQone) > 1 and abs(PQone_ele[0][1]-PQone_ele[1][1]) < 0.000001 and PQone_ele[0][2] > PQone_ele[1][2]:
                    alpha = PQone[1] # alpha is the second triangle
                    del PQone[1]
                    del PQone_ele[1]
                else:
                    alpha = PQone.popleft() # alpha is the first triangle, which is the same as alpha=PQone[0], then del PQone[0]
                    del PQone_ele[0]
                
                if num_unpaired_edges(alpha, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 0:
                    PQzero.append(alpha)
                else:
                    pair_alpha = pair_an_edge_from_PQzero(alpha, PQzero) # pair_alpha is an edge, [pair_alpha, alpha] will be a new paired vector
                    # if pair_alpha in PQzero:
                    PQzero.remove(pair_alpha) # remove pair_alpha from PQzero
                    # Forman_vec_pair.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    Forman_vec_pair_ET.append([pair_alpha, alpha]) # add the [pair_alpha, alpha], which is [edge, tri]
                    
                    beta_cell_temp_index_of_remove = []
                    beta_index_temp = 0
                    for cell_beta in LS_filtra: # this can be optimized
                        if cell_beta not in PQone and num_unpaired_edges(cell_beta, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                            PQone.append(cell_beta)
                            PQone_ele.append(LS_ele[beta_index_temp])
                            # remove cell_beta from LS_tri
                            # LS_tri.remove(cell_beta)
                            beta_cell_temp_index_of_remove.append(beta_index_temp)
                        beta_index_temp += 1
                    
                    # remove cell_beta from LS_filtra and LS_ele
                    for i in reversed(beta_cell_temp_index_of_remove):
                        del LS_filtra[i]
                        del LS_ele[i]
                    
                    PQone = sorted(PQone)
                    PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
            if len(PQzero) > 0:
                gama = PQzero.popleft() # gama is a critical simplex
                crit_cell.append(gama)
                
                gama_cell_temp_index_of_remove = []
                gama_index_temp = 0
                for cell_gama in LS_filtra: # this can be optimized
                    if cell_gama not in PQone and num_unpaired_edges(cell_gama, crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET) == 1:
                        PQone.append(cell_gama)
                        PQone_ele.append(LS_ele[gama_index_temp])
                        # remove cell_gama from LS_tri
                        # LS_tri.remove(cell_gama)
                        gama_cell_temp_index_of_remove.append(gama_index_temp)
                    gama_index_temp += 1
                    
                # remove cell_beta from LS_filtra and LS_ele
                for i in reversed(gama_cell_temp_index_of_remove):
                    del LS_filtra[i]
                    del LS_ele[i]
                        
                PQone = sorted(PQone)
                PQone = deque(PQone) # after sorting, PQone one will be a list instead of a queue, we need to reconstruct this queue
                
    return crit_cell, Forman_vec_pair_VE, Forman_vec_pair_ET

# StructType for Crit_cell        
add_Crit_cell_schema = StructType([
    StructField('Crit_cell',ArrayType(IntegerType()),True)
])

# StructType for VE pairs        
add_VE_schema = StructType([
    StructField("VE_pair_V", IntegerType(),True), 
    StructField('VE_pair_E',ArrayType(IntegerType()),True)
])

# StructType for ET pairs        
add_multi_ET_schema = StructType([
    StructField("ET_pair_E", ArrayType(IntegerType()),True), 
    StructField('ET_pair_T',ArrayType(IntegerType()),True)
])
 
# the whole StructType
get_Forman_schema = StructType([
    StructField("Crit_cell", ArrayType(ArrayType(IntegerType())), True),
    StructField("VE_pair", ArrayType(add_VE_schema), True),
    StructField("ET_pair", ArrayType(add_multi_ET_schema), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_Forman_udf = udf(get_Forman, get_Forman_schema)

In [30]:
df_Forman = df_LS_tri.withColumn("LS_Forman", get_Forman_udf(df_LS_tri.Ver, df_LS_tri.LS_edge, df_LS_tri.LS_filtra, df_LS_tri.LS_ele))
df_Forman = df_Forman.select(col("Ver"), col("VT_filtra"), col("LS_Forman").alias("Forman"))
df_Forman.printSchema()
print("The number of partitions for df_Forman:", df_Forman.rdd.getNumPartitions())
# df_Forman.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- ET_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)

The number of par

In [38]:
# num_df_Forman = df_Forman.count()
# print("number of rows in df_Forman:", num_df_Forman)

t_df_Forman = time.time()

time_Forman = t_df_Forman - t_df_LS_tri
print("time for computing Forman gradient:", time_Forman)

time for computing Forman gradient: 0.7583119869232178


In [39]:
# persist df_Forman
t_persist_0 = time.time()

df_Forman.persist(StorageLevel.DISK_ONLY)

t_persist_1 = time.time()
print("Total time cost for persisting df_Forman:", t_persist_1 - t_persist_0)

Total time cost for persisting df_Forman: 0.5005881786346436


# Extract critical vertices, edeges, and triangles

In [31]:
df_crit = df_Forman.select("Ver", col("Forman.Crit_cell").alias("Critical"))
print("number of partitions for df_crit:", df_crit.rdd.getNumPartitions())
df_crit.printSchema()
# df_crit.show()

number of partitions for df_crit: 20
root
 |-- Ver: integer (nullable = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [32]:
# split column Critical and obtain critical vertices, edges, and triangles
def get_crit_VET(crit_cell):
    if len(crit_cell) < 1: # no critical simplices
        return
    else:
        crit_ver = None
        crit_edge = []
        crit_tri = []
        for icrit_cell_temp in crit_cell:
            if len(icrit_cell_temp) == 1:
                crit_ver = icrit_cell_temp[0]
            if len(icrit_cell_temp) == 2:
                crit_edge.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 3:
                crit_tri.append(icrit_cell_temp)
        if len(crit_edge) == 0:
            crit_edge = None
        if len(crit_tri) == 0:
            crit_tri = None
        return crit_ver, crit_edge, crit_tri
        
# the whole StructType
get_crit_VET_schema = StructType([
    StructField("crit_ver", IntegerType(), True),
    StructField("crit_edge", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tri", ArrayType(ArrayType(IntegerType())), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_crit_VET_udf = udf(get_crit_VET, get_crit_VET_schema)

df_crit_VET = df_crit.withColumn("crit_cell", get_crit_VET_udf(df_crit.Critical))
print("number of partitions for df_crit_VET:", df_crit_VET.rdd.getNumPartitions())
df_crit_VET.printSchema()
# df_crit_VET.show()

number of partitions for df_crit_VET: 20
root
 |-- Ver: integer (nullable = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- crit_cell: struct (nullable = true)
 |    |-- crit_ver: integer (nullable = true)
 |    |-- crit_edge: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tri: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



In [42]:
'''
# critical vertices
df_crit_V = df_crit_VET.select(col("Ver"), col("crit_cell.crit_ver").alias("Min_ver")).na.drop()
df_crit_V.printSchema()
print("number of partitions for df_crit_V:", df_crit_V.rdd.getNumPartitions())
# df_crit_V.show()
'''

'\n# critical vertices\ndf_crit_V = df_crit_VET.select(col("Ver"), col("crit_cell.crit_ver").alias("Min_ver")).na.drop()\ndf_crit_V.printSchema()\nprint("number of partitions for df_crit_V:", df_crit_V.rdd.getNumPartitions())\n# df_crit_V.show()\n'

In [43]:
# num_crit_V = df_crit_V.count()
# print("number of critical vertices:", num_crit_V)

In [33]:
# critical edges
df_crit_E_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_edge").alias("Saddle_edge_init")).na.drop()

# there may be multiple saddles inside one lower star
df_crit_E = df_crit_E_init.select(col("Ver"), explode("Saddle_edge_init").alias("Saddle_edge"))
print("number of partitions for df_crit_E:", df_crit_E.rdd.getNumPartitions())

df_crit_E.printSchema()
print("number of partitions for df_crit_E:", df_crit_E.rdd.getNumPartitions())
# df_crit_E.show()

number of partitions for df_crit_E: 20
root
 |-- Ver: integer (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_E: 20


In [45]:
# persist df_Forman
t_persist_0 = time.time()

df_crit_E.persist(StorageLevel.DISK_ONLY)

t_persist_1 = time.time()
print("Total time cost for persisting df_crit_E:", t_persist_1 - t_persist_0)

Total time cost for persisting df_crit_E: 0.026323795318603516


In [46]:
# num_crit_E = df_crit_E.count()
# print("number of critical edges:", num_crit_E)

In [47]:
'''
# critical triangles
df_crit_T_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_tri").alias("Max_tri_init")).na.drop()

# critical triangles
df_crit_T = df_crit_T_init.select(col("Ver"), explode("Max_tri_init").alias("Max_tri")).na.drop()
print("number of partitions for df_crit_T:", df_crit_T.rdd.getNumPartitions())
df_crit_T.printSchema()
# df_crit_T.show()
'''

'\n# critical triangles\ndf_crit_T_init = df_crit_VET.select(col("Ver"), col("crit_cell.crit_tri").alias("Max_tri_init")).na.drop()\n\n# critical triangles\ndf_crit_T = df_crit_T_init.select(col("Ver"), explode("Max_tri_init").alias("Max_tri")).na.drop()\nprint("number of partitions for df_crit_T:", df_crit_T.rdd.getNumPartitions())\ndf_crit_T.printSchema()\n# df_crit_T.show()\n'

In [48]:
# num_crit_T = df_crit_T.count()
# print("number of critical triangles:", num_crit_T)

# Methods to build Graph_ET

### prepare nodes of a graph

In [34]:
# all triangles are nodes of graph V2
df_graph_V2_node = df_tri_order.select(col("tri_order"), col("tri"))
df_graph_V2_node.printSchema()
#df_graph_V2_node.show()

root
 |-- tri_order: integer (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [35]:
# add a special triangle [-1, -1, -1] as node -1
# this can be achieved by unioning two DataFrames
# columns_special = ['tri', 'V2_node']
value_special = [(-1, [-1, -1, -1])]
schema_special = StructType([
    StructField('tri_order', IntegerType(),True),
    StructField("tri", ArrayType(IntegerType()),True)    
])
df_special_V2_node = spark.createDataFrame(value_special, schema_special)
df_special_V2_node.printSchema()
#df_special_V2_node.show()

root
 |-- tri_order: integer (nullable = true)
 |-- tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)



### nodes of a graph

In [36]:
df_graph_V2_node = df_graph_V2_node.union(df_special_V2_node)
df_graph_V2_node_final = df_graph_V2_node.select(col("tri_order").alias("id"))
df_graph_V2_node_final.printSchema()
#df_graph_V2_node_final.show()

root
 |-- id: integer (nullable = true)



### prepare arcs of a graph

In [37]:
# Method 2: firstly extract df_Forman_ET_pair from df_Forman
df_Forman_ET_pair = df_Forman.select(col("Ver"), col("VT_filtra"), col("Forman.ET_pair").alias("ET_pair"))
df_Forman_ET_pair.printSchema()
# df_Forman_ET_pair.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- ET_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ET_pair_E: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- ET_pair_T: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [38]:
# retrieve co-boundary triangles
def get_co_tri(VT, ET_pair):
    if len(ET_pair) > 0:
        co_tri_pair = []
        num_ET_pair = len(ET_pair) # obtain the number of ET_pairs
        def Co_tri(e,t,vt):
            # e is sorted
            for t_temp in vt:
                # e0 = sorted([vt[0], vt[1]], reverse=True) # sort the edge in descending order
                e0 = [t_temp[0], t_temp[1]] # e0, e1, and e2 are in descending order by default
                e1 = [t_temp[0], t_temp[2]]
                e2 = [t_temp[1], t_temp[2]]
                t = [int(t[0]), int(t[1]), int(t[2])]
                t_temp = [int(t_temp[0]), int(t_temp[1]), int(t_temp[2])]
                # if e==e0 or e==e1 or e==e2 and t != t_temp:
                if e==e0 or e==e1 or e==e2:
                    if t != t_temp:
                        return t_temp # t_temp is a coboundary triangle of t in vt
                
        for i in range(num_ET_pair):
            ET_pair_e = ET_pair[i]['ET_pair_E'] # the i-th ET_pair
            ET_pair_t = ET_pair[i]['ET_pair_T']
            ET_pair_co_t = Co_tri(ET_pair_e, ET_pair_t, VT)
            if ET_pair_co_t is None: # len(ET_pair_co_t) is empty, boundary triangles
                ET_pair_co_t = [-1, -1, -1]
            co_tri_pair.append([ET_pair_co_t, ET_pair_t])
            
        return co_tri_pair
    
# the whole StructType
get_co_tri_schema = StructType([
    StructField("ET_pair_co_t", ArrayType(IntegerType()), True),
    StructField("ET_pair_t", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_co_tri_udf = udf(get_co_tri, ArrayType(get_co_tri_schema))

df_Forman_V2_edge = df_Forman_ET_pair.withColumn("ET_pair_inTri", get_co_tri_udf(df_Forman_ET_pair.VT_filtra, df_Forman_ET_pair.ET_pair))
df_Forman_V2_edge = df_Forman_V2_edge.select(col("Ver"), col("ET_pair_inTri"))
df_Forman_V2_edge.printSchema()
# df_Forman_V2_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- ET_pair_inTri: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ET_pair_co_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- ET_pair_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



### construct Graph_ET reversely, set src nodes as dst nodes, and dst nodes as src nodes

In [39]:
# explode df_Forman_V2_edge
df_Forman_V2_edge_split = df_Forman_V2_edge.select(explode(df_Forman_V2_edge.ET_pair_inTri).alias("ET_pair_src_dst"))

df_Forman_V2_edge_split = df_Forman_V2_edge_split.select(col("ET_pair_src_dst.ET_pair_co_t").alias("arc_dst"), col("ET_pair_src_dst.ET_pair_t").alias("arc_src")).na.drop()
df_Forman_V2_edge_split.printSchema()
# df_Forman_V2_edge_split.show()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- arc_src: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [40]:
# join two DataFrames by df_Forman_V2_edge_split.arc_src == df_Forman_V2_edge_split.arc_des
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_Forman_V2_edge_split_src = df_Forman_V2_edge_split.join(df_graph_V2_node, df_Forman_V2_edge_split.arc_src == df_graph_V2_node.tri, "left")
df_Forman_V2_edge_split_src = df_Forman_V2_edge_split_src.select(col("arc_dst"), col("tri_order").alias("src"))
df_Forman_V2_edge_split_src.printSchema()
#df_Forman_V2_edge_split_src.show()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- src: integer (nullable = true)



In [41]:
# join two DataFrames by df_Forman_V2_edge_split.arc_src == df_Forman_V2_edge_split.arc_des
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_Forman_V2_edge_split_dst = df_Forman_V2_edge_split_src.join(df_graph_V2_node, df_Forman_V2_edge_split_src.arc_dst == df_graph_V2_node.tri, "left")
df_Forman_V2_edge_split_dst = df_Forman_V2_edge_split_dst.select(col("src"), col("tri_order").alias("dst"))
# df_Forman_V2_edge_split_dst.printSchema()

### arcs of Graph_ET

In [42]:
df_graph_V2_edge_final = df_Forman_V2_edge_split_dst
df_graph_V2_edge_final.printSchema()
# df_graph_V2_edge_final.show()

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)



In [43]:
t_persist_0 = time.time()

# df_graph_V2_node_final.persist(StorageLevel.MEMORY_ONLY_SER)
# df_graph_V2_edge_final.persist(StorageLevel.MEMORY_ONLY_SER)

t_persist_1 = time.time()
print("Total time cost for persisting DataFrames:", t_persist_1 - t_persist_0)

Total time cost for persisting DataFrames: 3.0279159545898438e-05


In [59]:
# save df_VET_graph_node
# t_save_nodes_0 = time.time()
# file_df_graph_V2_node_final_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_graph_V2_node_final.parquet'
# 
# df_graph_V2_node_final.write.mode('overwrite').options(header='False', delimiter=',').parquet(file_df_graph_V2_node_final_hdfs)
# 
# t_save_nodes_1 = time.time()
# print("Time cost for saving df_graph_V2_node_final to a parquet file:", t_save_nodes_1-t_save_nodes_0)

In [60]:
# save df_VET_graph_node
# t_save_edges_0 = time.time()
# file_df_graph_V2_edge_final_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_graph_V2_edge_final.parquet'
# 
# df_graph_V2_edge_final.write.mode('overwrite').options(header='False', delimiter=',').parquet(file_df_graph_V2_edge_final_hdfs)
# 
# t_save_edges_1 = time.time()
# print("Time cost for saving df_graph_V2_edge_final to a parquet file:", t_save_edges_1-t_save_edges_0)

Time cost for saving df_graph_V2_edge_final to a parquet file: 3247.950574874878


In [61]:
assert(0 == 1)

AssertionError: 

### Construct a Graph_ET

In [43]:
graph_ET = GraphFrame(df_graph_V2_node_final, df_graph_V2_edge_final)

print("type of graph_V2:", type(graph_ET))

type of graph_V2: <class 'graphframes.graphframe.GraphFrame'>


In [44]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

In [45]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con = graph_ET.connectedComponents()
result_con.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)

time cost of connected components: 32.480376958847046


In [ ]:
# count the number of distinct connected components
# result_con_dist = result_con.dropDuplicates(['component'])
# num_connected = result_con_dist.count()
# print("number of distinct connected components:", num_connected)

In [ ]:
t_persist_result_con_0 = time.time()

# result_con.persist(StorageLevel.MEMORY_ONLY_SER)

t_persist_result_con_1 = time.time()
print("Total time cost for persisting result_con:", t_persist_result_con_1 - t_persist_result_con_0)

In [ ]:
'''
# save result_con
t0 = time.time()
file_result_con_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_result_con.parquet'

result_con.write.mode('overwrite').options(header='False', delimiter=',').parquet(file_result_con_hdfs)

t1 = time.time()
print("Time cost for saving result_con to a csv file:", t1-t0)
'''

### 1) left join result_con with saddles tri to identify the component that each saddles tri belongs to

In [46]:
df_VT = df_VV_VT.select('Ver', 'VT_filtra')

# df_VT.persist(StorageLevel.DISK_ONLY)

In [47]:
# join two DataFrames to obatin VE relation
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
df_crit_E_VT = df_crit_E.join(df_VT, df_crit_E.Ver == df_VT.Ver, "left")
df_crit_E_VT = df_crit_E_VT.select(df_crit_E.Ver, "Saddle_edge", "VT_filtra")
df_crit_E_VT.printSchema()
# df_crit_E_VT.show()

root
 |-- Ver: integer (nullable = true)
 |-- Saddle_edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- VT_filtra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [48]:
# retrieve co-boundary triangles
def get_saddle_edge_tri(Saddle_edge, VT):
    if len(VT) > 0:
        e = Saddle_edge
        tri_12 = []
        num_VT = len(VT) # obtain the number of ET_pairs
        for t_temp in VT:
            e0 = [t_temp[0], t_temp[1]] # e0, e1, and e2 are in descending order by default
            e1 = [t_temp[0], t_temp[2]]
            e2 = [t_temp[1], t_temp[2]]
            if e==e0 or e==e1 or e==e2:
                tri_12.append(t_temp)
            # if len(tri_12) == 2: # Maybe this triangle is at the boundary, so it has only one corresponding triangle in VT
                # break
        
        if len(tri_12) == 1:
            t_special = [-1,-1,-1]
            tri_12.append(t_special)
        
            
        return tri_12
    
# the whole StructType
get_saddle_edge_tri_schema = StructType([
    StructField("Saddle_edge_t1", ArrayType(IntegerType()), True),
    StructField("Saddle_edge_t2", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_saddle_edge_tri_udf = udf(get_saddle_edge_tri, ArrayType(ArrayType(IntegerType())))

df_crit_E_tri = df_crit_E_VT.withColumn("saddle_edge_Tri", get_saddle_edge_tri_udf(df_crit_E_VT.Saddle_edge, df_crit_E_VT.VT_filtra))
df_crit_E_tri = df_crit_E_tri.select(col("saddle_edge_Tri"))
df_crit_E_tri.printSchema()
# df_crit_E_tri.show()

root
 |-- saddle_edge_Tri: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [49]:
# obtain the saddle tri
df_crit_E2tri = df_crit_E_tri.select(explode("saddle_edge_Tri").alias("saddle_edge_Tri_split"))
# some nodes may belong to multiple saddles, we need to remove them
df_crit_E2tri = df_crit_E2tri.distinct()
df_crit_E2tri.printSchema()

root
 |-- saddle_edge_Tri_split: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [50]:
# left join df_crit_E2tri with df_graph_V2_node to obtain corresponding id of tri
df_crit_E2tri = df_crit_E2tri.join(df_graph_V2_node, df_crit_E2tri.saddle_edge_Tri_split == df_graph_V2_node.tri, "left")
df_crit_E2tri = df_crit_E2tri.select(col('tri_order').alias('saddle_edge_Tri_split_id'))
df_crit_E2tri.printSchema()

root
 |-- saddle_edge_Tri_split_id: integer (nullable = true)



In [ ]:
'''
# save df_crit_E2tri
t0 = time.time()
file_df_crit_E2tri_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_crit_E2tri.parquet'

df_crit_E2tri.write.mode('overwrite').options(header='False', delimiter=',').parquet(file_df_crit_E2tri_hdfs)

t1 = time.time()
print("Time cost for saving df_crit_E2tri to a csv file:", t1-t0)
'''

In [51]:
# identify the saddle tri that each component have
result_con_saddle = result_con.join(df_crit_E2tri,result_con.id==df_crit_E2tri.saddle_edge_Tri_split_id, "inner")

print("number of partitions for result_con_saddle:", result_con_saddle.rdd.getNumPartitions())
result_con_saddle.printSchema()
# result_con_saddle.show(30)

number of partitions for result_con_saddle: 20
root
 |-- id: integer (nullable = true)
 |-- component: long (nullable = true)
 |-- saddle_edge_Tri_split_id: integer (nullable = true)



In [52]:
# obtain all saddle tris for each connected component
unique_SdlTris_per_con = result_con_saddle.groupBy('component').agg(collect_list('id').alias('multi_SdlTris'))
unique_SdlTris_per_con.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTris: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [ ]:
t_persist_unique_SdlTris_per_con_0 = time.time()

# unique_SdlTris_per_con.persist(StorageLevel.MEMORY_ONLY)

t_persist_unique_SdlTris_per_con_1 = time.time()
print("Total time cost for persisting unique_SdlTris_per_con:", t_persist_unique_SdlTris_per_con_1 - t_persist_unique_SdlTris_per_con_0)

### 2) left join df_graph_V2_edge_final with result_con to identify the connected component that each arc belongs to

In [53]:
# left join df_graph_V1_arc with result_con to identify which component that each arc belongs to
df_G_V2_arc_component = df_graph_V2_edge_final.join(result_con,df_graph_V2_edge_final.src==result_con.id, "left").drop('id')
df_G_V2_arc_component.printSchema()

root
 |-- src: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- component: long (nullable = true)



In [ ]:
'''
df_G_V2_arc_component_col = df_G_V2_arc_component.collect()

num_None_src = 0
num_None_dst = 0
num_neg1_src = 0
num_neg1_dst = 0
for i in range(len(df_G_V2_arc_component_col)):
    if df_G_V2_arc_component_col[i][0] == None:
        num_None_src += 1
    if df_G_V2_arc_component_col[i][1] == None:
        num_None_dst += 1
        
    if df_G_V2_arc_component_col[i][0] == -1:
        num_neg1_src += 1
    if df_G_V2_arc_component_col[i][1] == -1:
        num_neg1_dst += 1
        
print("num_None_src:",num_None_src)
print("num_None_dst:",num_None_dst)
print("num_neg1_src:",num_neg1_src)
print("num_neg1_dst:",num_neg1_dst)
'''

### create a dictionary for each connected component

In [54]:
from pyspark.sql.functions import create_map

In [55]:
# convert each arc to a dictionary
df_G_V2_arc_component_dict = df_G_V2_arc_component.withColumn("arcs", create_map(col('src'), col('dst'))).drop('src','dst')

# combine dictionaries belonging to an identical component to one list
df_G_V2_arc_component_dict_gpy = df_G_V2_arc_component_dict.groupBy('component').agg(collect_list('arcs').alias('subgraphs'))

# left join df_G_V1_arc_component_dict_gpy with unique_SdlPts_per_con to obtain SdlPts belonging to each component
df_G_V2_arc_component_dict_gpy = df_G_V2_arc_component_dict_gpy.join(unique_SdlTris_per_con,df_G_V2_arc_component_dict_gpy.component==unique_SdlTris_per_con.component, "left").select(df_G_V2_arc_component_dict_gpy.component, 'multi_SdlTris', 'subgraphs')
df_G_V2_arc_component_dict_gpy.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTris: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- subgraphs: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: integer
 |    |    |-- value: integer (valueContainsNull = true)



In [ ]:
t_persist_df_G_V2_arc_component_dict_gpy_0 = time.time()

# df_G_V2_arc_component_dict_gpy.persist(StorageLevel.MEMORY_ONLY_SER)

t_persist_df_G_V2_arc_component_dict_gpy_1 = time.time()
print("Total time cost for persisting df_G_V2_arc_component_dict_gpy:", t_persist_df_G_V2_arc_component_dict_gpy_1 - t_persist_df_G_V2_arc_component_dict_gpy_0)

In [ ]:
# unpersist
# result_con.unpersist()
# df_graph_V2_node_final.unpersist()
# df_graph_V2_edge_final.unpersist()
# df_crit_E.unpersist()
# df_Forman.unpersist()
unique_SdlTris_per_con.unpersist()

In [ ]:
# save df_G_V2_arc_component_dict_gpy
t0 = time.time()
file_df_G_V2_arc_component_dict_gpy_hdfs = 'SigSpatial_data' + '/' + tin_filename + '_G_V2_arc_component_dict_gpy.parquet'

# df_G_V2_arc_component_dict_gpy.write.mode('overwrite').options(header='False', delimiter=',').parquet(file_df_G_V2_arc_component_dict_gpy_hdfs)

t1 = time.time()
print("Time cost for saving df_G_V2_arc_component_dict_gpy to a csv file:", t1-t0)

In [ ]:
assert(1 == 0)

In [56]:
def bfs_rdd(component, multi_SdlTris, subgraphs):
    # Perform a breadth-first search on a single graph
    
    # subgraphs is a list of dictionaries, we need to convert it to a single dictionary
    subgraphs_dict = {}
    # merge all sub-dictionaries to one single dictionary
    for d in subgraphs:
        subgraphs_dict = {**subgraphs_dict, **d}
        
    visited = set()
    queue = multi_SdlTris
    while queue:
        node = queue.pop(0)
        print("node:",node)
        if node not in visited:
            visited.add(node)
            neighbors = subgraphs_dict.get(node, [])
            if neighbors != []:
                queue.append(neighbors)
                
    visited = list(visited)
    return component, visited

In [57]:
# Use mapPartitions to apply the bfs function to each partition of the RDD
# Each partition corresponds to a single graph that is processed by a single executor
t_V2_0 = time.time()

df_V2_result = df_G_V2_arc_component_dict_gpy.rdd.map(lambda partition: bfs_rdd(partition['component'], partition['multi_SdlTris'], partition['subgraphs'])).toDF()
# V2_result_col = V2_result.collect()
num_df_V2_result = df_V2_result.count()
print("num_df_V2_result:", num_df_V2_result)

t_V2_1 = time.time()
print("Time cost to extract V2-Paths:", t_V2_1 - t_V2_0)

num_df_V2_result: 2
Time cost to extract V2-Paths: 9.457555532455444


In [58]:
df_V2_result_col = df_V2_result.collect()

In [60]:
df_V2_result_col

[Row(_1=400, _2=[512, 513, 896, 897, 654, 1040, 785, 1041, 787, 786, 789, 657, 792, 793, 794, 795, 796, 797, 414, 798, 799, 417, 802, 801, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 560, 944, 561, 945, 1088, 1089, 704, 705, 462, 463, 846, 849, 606, 736, 993, 992, 739, 740, 609, 742, 743, 745, 750, 1136, 1137, 752, 755, 753, 510]),
 Row(_1=-1, _2=[128, 129, 770, 771, 772, 773, 774, 775, 776, 768, 10, 778, 12, 780, 14, 15, 782, 17, 783, 16, 13, 19, 271, 18, 21, 769, 32, 33, 176, 177, 318, 321, 270, 779, 81, 80, 726, 729, 781, 222, 224, 225, 368, 369, -1])]

In [64]:
len(df_V2_result_col[1][1])

45

In [59]:
df_V2_result_col

[Row(_1=400, _2=[512, 513, 896, 897, 654, 1040, 785, 1041, 787, 786, 789, 657, 792, 793, 794, 795, 796, 797, 414, 798, 799, 417, 802, 801, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 560, 944, 561, 945, 1088, 1089, 704, 705, 462, 463, 846, 849, 606, 736, 993, 992, 739, 740, 609, 742, 743, 745, 750, 1136, 1137, 752, 755, 753, 510]),
 Row(_1=-1, _2=[128, 129, 770, 771, 772, 773, 774, 775, 776, 768, 10, 778, 12, 780, 14, 15, 782, 17, 783, 16, 13, 19, 271, 18, 21, 769, 32, 33, 176, 177, 318, 321, 270, 779, 81, 80, 726, 729, 781, 222, 224, 225, 368, 369, -1])]

In [ ]:
print("time cost of connected components:",t_con)

print("Time cost to extract V2-Paths:", t_V2_1 - t_V2_0)

In [ ]:
# print("number of distinct connected components:", num_connected)

In [ ]:
# num_crit_T = df_crit_T.count()
# print("number of critical triangles:", num_crit_T)

In [ ]:
print("River road")

In [ ]:
assert(1 == 0)

In [ ]:
t_save_V2_Paths_0 = time.time()

V2_path = 'SigSpatial_data' + '/' + tin_filename + '_' + 'V2_Paths' + '.parquet'
df_V2_result.write.format("parquet").mode("overwrite").save(V2_path)

t_save_V2_Paths_1 = time.time()
t_save_V2_Paths = t_save_V2_Paths_1 - t_save_V2_Paths_0
print("time for saving V2_Paths:", t_save_V2_Paths)

In [ ]:
import requests

def notify(mgs=""):
    token = "bce6ba"
   
    url = f"https://telepush.dev/api/messages/{token}"
    payload = {"text": mgs}
    response = requests.post(url, json=payload)
   
    print(response.status_code)

msg_tele = spark_app_name + 'ran correctly!'
notify(msg_tele)